<a href="https://colab.research.google.com/github/lucifernandes/dadosSociaisMLP_2/blob/master/RN_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pré-processamento

In [ ]:
import pandas as pd
base = pd.read_csv('/content/drive/My Drive/Bases/base_v01.csv', sep=',', decimal=',')
base = base.dropna(how="any") 
base = base.reset_index(drop=True)
base.head(20)

#------------------------------- índice Autoritarismo ------------------------------------------------
#Sentenças do 4º bloco do questionário
baseI = base.iloc[:, 68:85]  
#Adicionando à base a coluna 'IndAuto' com os valores dos índices de cada amostra                   
A = baseI.mean(axis=1)

#Cálculo do índice médio  
mediana = A.median()
#Para valores abaixo ou igual a média, recebem valor 0. Para valores acima da média, recebem valor 1
A = A.values
print(mediana)

for i in range(1757):
  if A[i] < mediana:
    A[i] = 0
  else:
    A[i] = 1

base['IndAuto'] = A

base['IndAuto'] = (base['IndAuto']).astype(int)
base['renda'] = (base['renda']).astype(int)
#base.groupby(by='IndAuto').size()
#print(baseI.mean(axis=1))

previsores = base.iloc[:, 0:68].values
classe = base.iloc[:, 85].values

#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler() #Padronização
#previsores = scaler.fit_transform(previsores)

#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#previsores = scaler.fit_transform(previsores)
print(len(previsores))

4.411764705882353
1757


In [ ]:
previsores.head()

## Abordagem Validação Cruzada

In [ ]:
import numpy as np
#a = np.zeros(5)
#b = np.zeros(shape=(previsores.shape[0], 1))

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

resultados20 = []
for i in range(20):
  kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = i)
  resultados1 = []

  for indice_treinamento, indice_teste in kfold.split(previsores,
                                                        np.zeros(shape=(previsores.shape[0], 1))):
    #Classificador
    #classificador = Perceptron(max_iter=1000, tol=1e-4, n_iter_no_change=30, random_state=i) #Perceptron
    classificador = MLPClassifier(hidden_layer_sizes=(35), max_iter=1000, activation='logistic', #MLP
                                  solver='adam', n_iter_no_change=30)
    
    classificador.fit(previsores[indice_treinamento], classe[indice_treinamento]) 
    
    previsoes = classificador.predict(previsores[indice_teste])
    precisao = accuracy_score(classe[indice_teste], previsoes)
    resultados1.append(precisao)

  resultados1 = np.asarray(resultados1)
  media = resultados1.mean()
  resultados20.append(media)

  print(str(media).replace('.', ','))
 

  #resultados20 = np.asarray(resultados30)

#for i in range(20):
   # print(str(resultados20[i]).replace('.', ','))

#print('Acurácia:\t', resultados20.mean())
#print(media)

## Abordagem Hold out

### PCA

In [ ]:
resultados20 = []
for i in range(20):
  from sklearn.model_selection import train_test_split
  previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.1, random_state=0)

  from sklearn.decomposition import PCA
  pca = PCA(n_components = 40)
  previsores_treinamento = pca.fit_transform(previsores_treinamento)
  previsores_teste = pca.transform(previsores_teste)
  componentes = pca.explained_variance_ratio_
  #print(componentes.sum())
  #print(len(componentes))

  from sklearn.linear_model import Perceptron
  from sklearn.neural_network import MLPClassifier
  #Classificador
  #classificador = Perceptron(max_iter=1000, tol=1e-4, n_iter_no_change=30, random_state=i) #Perceptron
  classificador = MLPClassifier(hidden_layer_sizes=(35), max_iter=1000, activation='logistic', #MLP
                                  solver='sgd', n_iter_no_change=30)

  classificador.fit(previsores_treinamento, classe_treinamento)
  previsoes = classificador.predict(previsores_teste)

  from sklearn.metrics import confusion_matrix, accuracy_score
  precisao = accuracy_score(classe_teste, previsoes)
  matriz = confusion_matrix(classe_teste, previsoes)
  #print(precisao)
  resultados20.append(precisao)

for i in range(20):
  print(str(resultados20[i]).replace('.', ','))


### Sem PCA

In [ ]:
resultados20 = []
for i in range(20):
  from sklearn.model_selection import train_test_split
  previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.10, random_state=i)
      #print(previsores_teste)

  from sklearn.linear_model import Perceptron
  from sklearn.neural_network import MLPClassifier
      #Classificador
  #classificador = Perceptron(max_iter=5000, tol=1e-4, n_iter_no_change=30, random_state=i) #Perceptron
  classificador = MLPClassifier(hidden_layer_sizes=(35), max_iter=2500, activation='identity', #MLP
                                  solver='sgd', n_iter_no_change=30, beta_1=0.999, learning_rate='adaptive', learning_rate_init=0.01 )

  classificador.fit(previsores_treinamento, classe_treinamento)
  previsoes = classificador.predict(previsores_teste)

  from sklearn.metrics import confusion_matrix, accuracy_score
  precisao = accuracy_score(classe_teste, previsoes)
  matriz = confusion_matrix(classe_teste, previsoes)
    #print(precisao)
    #print(previsoes)

  resultados20.append(precisao)
  #print(str(precisao).replace('.', ','))

for i in range(20):
  print(str(resultados20[i]).replace('.', ','))

#print(precisao)


0,7045454545454546
0,6306818181818182
0,6136363636363636
0,6420454545454546
0,6590909090909091
0,5852272727272727
0,5625
0,625
0,6534090909090909
0,6079545454545454
0,6306818181818182
0,6704545454545454
0,7159090909090909
0,5738636363636364
0,5625
0,6022727272727273
0,625
0,6477272727272727
0,6079545454545454
0,5909090909090909
